# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation
from madminer.plotting import plot_fisher_information_contours_2d

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'
result_dir = base_dir + 'data/results/wgamma/'

## Settings

In [4]:
weight_settings = [None, 1., 10., 100.]
weight_setting_filenames = ['', '_w1', '_w10', '_w100']

## FisherInformation instance

In [5]:
fisher = FisherInformation(sample_dir + 'samples.h5', debug=True)

07:23  
07:23  ------------------------------------------------------------
07:23  |                                                          |
07:23  |  MadMiner v2018.10.26                                    |
07:23  |                                                          |
07:23  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
07:23  |                                                          |
07:23  ------------------------------------------------------------
07:23  
07:23  Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
07:23  Found 2 parameters:
07:23     cWWW (LHA: dim6 1, maximal power in squared ME: (2,), range: (-0.1, 0.1))
07:23     cWWWtilde (LHA: dim6 2, maximal power in squared ME: (2,), range: (-0.1, 0.1))
07:23  Found 6 benchmarks:
07:23     sm: cWWW = 0.00e+00, cWWWtilde = 0.00e+00
07:23     morphing_basis_vector_1: cWWW = -9.39e-02, cWWWtilde = 0.07
07:23     morphing_basis_vector_2: cWWW = 0.

## Truth-level info

In [6]:
fisher_info_parton = fisher.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton.npy', fisher_info_parton)


## Rate-only info

In [7]:
fisher_info_xsec = fisher.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec.npy', fisher_info_xsec)


## Full Fisher info (ML)

In [8]:
means, covs = fisher.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all',
    unweighted_x_sample_file = sample_dir + 'test/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full{}.npy'.format(weight_filename), cov)

07:26  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/ensemble.json
07:26    Found ensemble with 20 estimators and expectations [[-0.01433866 -0.00798974]
 [-0.04412725  0.02471908]
 [-0.06542948  0.0835643 ]
 [-0.02332889 -0.00558355]
 [ 0.02657468  0.0145047 ]
 [ 0.11239411 -0.03549213]
 [ 0.00559056 -0.00027227]
 [-0.07079547  0.10741747]
 [-0.00762126  0.01911212]
 [ 0.00617709  0.01502894]
 [ 0.00371833 -0.03548462]
 [ 0.01541766 -0.01456542]
 [ 0.06796021  0.00998996]
 [ 0.02345792 -0.02461899]
 [-0.00599976  0.01418786]
 [ 0.01237708 -0.0282448 ]
 [-0.02025609  0.01516213]
 [-0.00509163  0.00717454]
 [ 0.00775625 -0.04099903]
 [-0.02500463 -0.03304806]]
07:26  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_0_settings.json
07:26    Found method sally, 29 observables, 2 parameters, (100, 100) hidden layers, tanh activation 

07:26    Found method sally, 29 observables, 2 parameters, (100, 100) hidden layers, tanh activation function, features None
07:26  Loading state dictionary from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_17_state_dict.pt
07:26  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_18_settings.json
07:26    Found method sally, 29 observables, 2 parameters, (100, 100) hidden layers, tanh activation function, features None
07:26  Loading state dictionary from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_18_state_dict.pt
07:26  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_19_settings.json
07:26    Found method sally, 29 observables, 2 parameters, (100, 100) hidden layers, tanh activation function, features No

## Info in observables (ML)

In [9]:
means, covs = fisher.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_deltaphi',
    unweighted_x_sample_file = sample_dir + 'test/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_deltaphi{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_deltaphi{}.npy'.format(weight_filename), cov)

07:32  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/ensemble.json
07:32    Found ensemble with 20 estimators and expectations [[ 0.00875856  0.00459889]
 [-0.0003045   0.00149938]
 [ 0.00439043 -0.00443414]
 [-0.0059252   0.01023495]
 [-0.00315193 -0.00054421]
 [ 0.00304334  0.00724456]
 [ 0.00237222 -0.00301501]
 [ 0.0007224   0.00481131]
 [-0.00355629  0.00685876]
 [ 0.00090163 -0.00755925]
 [ 0.0026124  -0.01008341]
 [ 0.00877647  0.00730935]
 [ 0.00096675 -0.0042282 ]
 [-0.00017658 -0.00820993]
 [-0.00691816 -0.00035836]
 [-0.0018715   0.0006327 ]
 [ 0.00078935 -0.00366219]
 [ 0.00461002  0.0142154 ]
 [ 0.00450602  0.0049802 ]
 [ 0.00160791  0.00054987]]
07:32  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/estimator_0_settings.json
07:32    Found method sally, 1 observables, 2 parameters, (100, 100) hidden layers, tanh ac

07:32  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/estimator_17_settings.json
07:32    Found method sally, 1 observables, 2 parameters, (100, 100) hidden layers, tanh activation function, features [20]
07:32  Loading state dictionary from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/estimator_17_state_dict.pt
07:32  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/estimator_18_settings.json
07:32    Found method sally, 1 observables, 2 parameters, (100, 100) hidden layers, tanh activation function, features [20]
07:32  Loading state dictionary from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/estimator_18_state_dict.pt
07:32  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgam

In [10]:
means, covs = fisher.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_met',
    unweighted_x_sample_file = sample_dir + 'test/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_met{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_met{}.npy'.format(weight_filename), cov)

07:38  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_met/ensemble.json
07:38    Found ensemble with 20 estimators and expectations [[-0.03217312  0.04683511]
 [ 0.00696393  0.03543703]
 [ 0.02575764  0.01514627]
 [-0.00298495  0.04633479]
 [ 0.07877649 -0.10351486]
 [ 0.03489412 -0.04659313]
 [-0.0472172  -0.01366843]
 [ 0.02494288 -0.11711625]
 [ 0.02242893  0.02772917]
 [-0.0252324  -0.03115297]
 [-0.01470638 -0.0187749 ]
 [ 0.0214741  -0.02624243]
 [ 0.01414475 -0.04351106]
 [-0.04331799 -0.05038221]
 [ 0.00418201 -0.02652313]
 [ 0.06079252  0.00274209]
 [-0.01953946 -0.02757395]
 [-0.07849615  0.00724561]
 [ 0.0044371  -0.00227884]
 [ 0.04675819  0.01130327]]
07:38  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_met/estimator_0_settings.json
07:38    Found method sally, 1 observables, 2 parameters, (100, 100) hidden layers, tanh activation f

07:38    Found method sally, 1 observables, 2 parameters, (100, 100) hidden layers, tanh activation function, features [0]
07:38  Loading state dictionary from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_met/estimator_17_state_dict.pt
07:38  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_met/estimator_18_settings.json
07:38    Found method sally, 1 observables, 2 parameters, (100, 100) hidden layers, tanh activation function, features [0]
07:38  Loading state dictionary from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_met/estimator_18_state_dict.pt
07:38  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_met/estimator_19_settings.json
07:38    Found method sally, 1 observables, 2 parameters, (100, 100) hidden layers, tanh activation function, features [0]
07:

## Histogram info

In [11]:
filenames = ['met', 'ptl', 'pta', 'deltaphi_lv', 'deltaphi_la']
observables = ['et_miss', 'pt_l1', 'pt_a1', 'deltaphi_l1_met', 'deltaphi_l1_a1']
obs_mins = [0., 0., 0., 0., 0.]
obs_maxs = [400., 400., 400., 6., 6.]
bins = 20

In [12]:
for filename, obs, obs_min, obs_max in zip(filenames, observables, obs_mins, obs_maxs):
    info = fisher.calculate_fisher_information_hist1d(
                theta=[0.,0.],
                luminosity=300000.,
                observable=obs,
                nbins=bins,
                histrange=(obs_min,obs_max)
           )
    
    np.save(result_dir + 'information_histo_{}.npy'.format(filename), info)
    

/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:910: RuntimeWarning: divide by zero encountered in double_scalars
  * np.tensordot(dsigma.T[i_event], dsigma.T[i_event], axes=0)
/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:910: RuntimeWarning: invalid value encountered in multiply
  * np.tensordot(dsigma.T[i_event], dsigma.T[i_event], axes=0)
